In [1]:
import copy
import torch
from torch import nn

In [2]:
from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from models.prophecy_paper_model import ProphecyPaperNetwork

Instructions for updating:
non-resource variables are not supported in the long term


### Test DP

In [3]:
dp_model = ProphecyPaperNetwork()
dp = MarabouCoreDP()

In [7]:
activation = {1: ['ON', 'OFF'], 3: ['ON', 'OFF']}
dp.solve(activation, dp_model, 0)

['unsat',
 {},

In [10]:
activation = {1: ['ON', 'OFF'], 3: ['--', '--']}
dp.solve(activation, dp_model, 0)

['unsat',
 {},

In [11]:
activation = {1: ['ON', '--'], 3: ['--', '--']}
dp.solve(activation, dp_model, 0)

['sat',
 {0: -49.9999995,
  1: -50.0000005,
  2: 1e-06,
  3: -100.0,
  4: 1e-06,
  5: 0.0,
  6: 5e-07,
  7: -5e-07,
  8: 5e-07,
  9: 5e-07,
  10: -0.0,
  11: -0.0},

### Test Iterative Relaxation

In [12]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_property = iterative_relaxation.call(ir_model, [[1, -1]], 0)
print(f"input property: {input_property}")

input property: [{1: ['ON', 'OFF'], 3: ['--', '--']}]


In [13]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_property = iterative_relaxation.call(ir_model, [[1, -1]], 1)
print(f"input property: {input_property}")

input property: [{1: ['ON', 'OFF'], 3: ['ON', 'OFF']}, 1]


### Manual example

In [ ]:
import sys
import numpy as np
sys.path.append('/Users/khoanguyen-cp/gmu/Marabou')

from maraboupy import MarabouCore
from maraboupy.Marabou import createOptions
inputQuery = MarabouCore.InputQuery()
inputQuery.setNumberOfVariables(12)

# boundaries
for i in [0,1]:
  inputQuery.setLowerBound(i,-10)
  inputQuery.setUpperBound(i,10)

for i in [2,3,6,7,10,11]:
  inputQuery.setLowerBound(i,-100)
  inputQuery.setUpperBound(i,100)

# activated relu: > 0
for i in [4]:
  inputQuery.setLowerBound(i,1e-6)
  inputQuery.setUpperBound(i,100)
  
# unactivated relu:
for i in []:
  inputQuery.setLowerBound(i,0)
  inputQuery.setUpperBound(i,0)
  
# unconstrained
for i in [5, 8, 9]:
  inputQuery.setLowerBound(i,0)
  inputQuery.setUpperBound(i,100)

# output constraint
equation_type = MarabouCore.Equation.EquationType(2) # type 2 is Less than or equal (LE inequality)
equation = MarabouCore.Equation(equation_type) 
equation.addAddend(1, 10)
equation.addAddend(-1, 11)
equation.setScalar(0)
disjunction = []
disjunction.append([equation])
MarabouCore.addDisjunctionConstraint(inputQuery, disjunction)
# inputQuery.addEquation(equation)

# layer 1, 
# node1,1b
# x0-x1=x2 
equation=MarabouCore.Equation()
equation.addAddend(1,0)
equation.addAddend(-1,1)
equation.addAddend(-1,2)
equation.setScalar(0)
inputQuery.addEquation(equation)

#node1,2b
#x0+x1=x3 
equation=MarabouCore.Equation()
equation.addAddend(1,0)
equation.addAddend(1,1)
equation.addAddend(-1,3)
equation.setScalar(0)
inputQuery.addEquation(equation)

# layer 2
#node2,1b
#0.5x4-0.2x5=x6
equation=MarabouCore.Equation()
equation.addAddend(0.5,4)
equation.addAddend(-0.2,5)
equation.addAddend(-1,6)
equation.setScalar(0)
inputQuery.addEquation(equation)

#node2,2b
#-0.5x4+0.1x5=x7
equation=MarabouCore.Equation()
equation.addAddend(-0.5,4)
equation.addAddend(0.1,5)
equation.addAddend(-1,7)
equation.setScalar(0)
inputQuery.addEquation(equation)

# layer output
#node3,1b
#x8-x9=x10
equation=MarabouCore.Equation()
equation.addAddend(1,8)
equation.addAddend(-1,9)
equation.addAddend(-1,10)
equation.setScalar(0)
inputQuery.addEquation(equation)

#node3,2b
#-x8+x9=x11
equation=MarabouCore.Equation()
equation.addAddend(-1,8)
equation.addAddend(1,9)
equation.addAddend(-1,11)
equation.setScalar(0)
inputQuery.addEquation(equation)

# relu constraints
MarabouCore.addReluConstraint(inputQuery, 2,4) #x4=relu(x2)
MarabouCore.addReluConstraint(inputQuery, 3,5) #x5=relu(x3)
MarabouCore.addReluConstraint(inputQuery, 6,8)#x8=relu(x6)
MarabouCore.addReluConstraint(inputQuery, 7,9)#x9=relu(x7)

# solve
options = createOptions(verbosity=0)
MarabouCore.solve(inputQuery, options, "")